In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jehanbhathena/weather-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mustafamarzouk
Your Kaggle Key: ··········


100%|██████████| 587M/587M [00:18<00:00, 33.9MB/s]


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import os
import glob
import random
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.models import Model
from sklearn import preprocessing
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image
from PIL import Image
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import pickle

In [5]:
class_names = ['dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning' , 'rain', 'rainbow', 'rime', 'sandstorm', 'snow']
folder_names = ["0",     "1",      "2",     "3",    "4",      "5",        "6",      "7",      "8",      "9",      "10" ] 

In [6]:
def read_images_from_dir(directory):
    images = []
    for folder in folder_names:
        folder_path = os.path.join(directory, class_names[int(folder)])
        print(folder_path)
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            print(image_path)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Could not read image file {image_path}")
                continue
            print(image.shape)
            image = cv2.resize(image, (96, 96))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert from BGR to RGB
            images.append((image, int(folder)))  # Store the image and its class label (0 or 1) as a tuple
    return images

In [7]:
all_data = read_images_from_dir('./weather-dataset/dataset/')

Streaming output truncated to the last 5000 lines.
(273, 487, 3)
./weather-dataset/dataset/rainbow/0673.jpg
(557, 549, 3)
./weather-dataset/dataset/rainbow/0635.jpg
(165, 179, 3)
./weather-dataset/dataset/rainbow/0780.jpg
(474, 1024, 3)
./weather-dataset/dataset/rainbow/0733.jpg
(160, 336, 3)
./weather-dataset/dataset/rainbow/0761.jpg
(205, 398, 3)
./weather-dataset/dataset/rainbow/0612.jpg
(275, 537, 3)
./weather-dataset/dataset/rainbow/0810.jpg
(162, 355, 3)
./weather-dataset/dataset/rainbow/0687.jpg
(755, 1007, 3)
./weather-dataset/dataset/rainbow/0649.jpg
(168, 300, 3)
./weather-dataset/dataset/rainbow/0679.jpg
(221, 306, 3)
./weather-dataset/dataset/rainbow/0757.jpg
(310, 280, 3)
./weather-dataset/dataset/rainbow/0714.jpg
(158, 182, 3)
./weather-dataset/dataset/rainbow/0734.jpg
(308, 610, 3)
./weather-dataset/dataset/rainbow/0765.jpg
(215, 395, 3)
./weather-dataset/dataset/rainbow/0759.jpg
(578, 800, 3)
./weather-dataset/dataset/rainbow/0746.jpg
(206, 383, 3)
./weather-dataset/dat

In [8]:
class_images = {class_name: [] for class_name in class_names}

for image, class_label in all_data:
    class_images[class_names[class_label]].append(image)

class_0 = np.array(class_images['dew'])
class_1 = np.array(class_images['fogsmog'])
class_2  = np.array(class_images['frost'])
class_3 = np.array(class_images['glaze'])
class_4 = np.array(class_images['hail'])
class_5 = np.array(class_images['lightning'])
class_6 = np.array(class_images['rain'])
class_7 = np.array(class_images['rainbow'])
class_8 = np.array(class_images['rime'])
class_9 = np.array(class_images['sandstorm'])
class_10 = np.array(class_images['snow'])


print(len(class_0))
print(len(class_1))
print(len(class_2))
print(len(class_3))

698
850
475
639


In [9]:
data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.4),
])

augmented_images_0 = []  
augmented_images_1 = []
augmented_images_2 = []
augmented_images_3 = []
augmented_images_4 = []
augmented_images_5 = []
augmented_images_6 = []
augmented_images_7 = []
augmented_images_8 = []
augmented_images_9 = []
augmented_images_10 = []


for i in range(2):
     # Append augmented images to the new array
    class0_augment = data_augmentation(class_0)
    augmented_images_0.extend(class0_augment) 
    
    class1_augment = data_augmentation(class_1)
    augmented_images_1.extend(class1_augment)
    
    class3_augment = data_augmentation(class_3)
    augmented_images_3.extend(class3_augment)
    
    class9_augment = data_augmentation(class_9)
    augmented_images_9.extend(class9_augment)
    
    class10_augment = data_augmentation(class_10)
    augmented_images_10.extend(class10_augment)
    
    class4_augment = data_augmentation(class_4)
    augmented_images_4.extend(class4_augment)
    
    class6_augment = data_augmentation(class_6)
    augmented_images_6.extend(class6_augment)
    
    
    
for i in range(3):    
    class2_augment = data_augmentation(class_2)
    augmented_images_2.extend(class2_augment)
    
    class5_augment = data_augmentation(class_5)
    augmented_images_5.extend(class5_augment)
    

    
for i in range(5):
    class7_augment = data_augmentation(class_7)
    augmented_images_7.extend(class7_augment)
    

print('0: ', len(augmented_images_0))
print('1: ', len(augmented_images_1))
print('2: ', len(augmented_images_2))
print('3: ',len(augmented_images_3))
print('4: ',len(augmented_images_4))
print('5: ',len(augmented_images_5))
print('6: ',len(augmented_images_6))
print('7: ',len(augmented_images_7))
print('8: ',len(class_8))
print('9: ',len(augmented_images_9))
print('10: ',len(augmented_images_10))

0:  1396
1:  1700
2:  1425
3:  1278
4:  1182
5:  1131
6:  1052
7:  1160
8:  1160
9:  1384
10:  1240


In [10]:
all_augmented_data = []

all_augmented_data.extend(augmented_images_0)
all_augmented_data.extend(augmented_images_1)
all_augmented_data.extend(augmented_images_2)
all_augmented_data.extend(augmented_images_3)
all_augmented_data.extend(augmented_images_4)
all_augmented_data.extend(augmented_images_5)
all_augmented_data.extend(augmented_images_6)
all_augmented_data.extend(augmented_images_7)
all_augmented_data.extend(augmented_images_8)
all_augmented_data.extend(augmented_images_9)
all_augmented_data.extend(augmented_images_10)

print(len(all_augmented_data))

12948


In [11]:
y_aug = []

for i in range(len(augmented_images_0)):
    y_aug.append(0)
    
for i in range(len(augmented_images_1)):
    y_aug.append(1)
    
for i in range(len(augmented_images_2)):
    y_aug.append(2)    
    
for i in range(len(augmented_images_3)):
    y_aug.append(3)    
    
for i in range(len(augmented_images_4)):
    y_aug.append(4)    
    
for i in range(len(augmented_images_5)):
    y_aug.append(5)    
    
for i in range(len(augmented_images_6)):
    y_aug.append(6)    
    
for i in range(len(augmented_images_7)):
    y_aug.append(7)
    
for i in range(len(augmented_images_8)):
    y_aug.append(8)

for i in range(len(augmented_images_9)):
    y_aug.append(9)
    
for i in range(len(augmented_images_10)):
    y_aug.append(10)       
    
print(len(y_aug))

12948


In [12]:
all_augmented_data = np.array(all_augmented_data)
y_aug = np.array(y_aug)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(all_augmented_data, y_aug, test_size=0.2, random_state=42, shuffle=True)

In [14]:
train_features = x_train.reshape(x_train.shape[0], -1)
test_features = x_test.reshape(x_test.shape[0], -1)
train_features.shape

(10358, 27648)

In [15]:
classifiers = [
    ('SVM', SVC(), {'kernel': 'rbf', 'C': 3}),
    ('Random Forest', RandomForestClassifier(), {'n_estimators': 100}),
    ('KNN', KNeighborsClassifier(), {'n_neighbors': 3}),
    ('Logistic Regression', LogisticRegression(), {'multi_class': 'ovr', 'solver': 'liblinear'}),
    ('Decision Tree', DecisionTreeClassifier(), {'max_depth': None})
]

In [ ]:
for name, model, params in classifiers:
    print(f"Training {name}...")
    model.set_params(**params)
    model.fit(train_features, y_train)
    
    y_pred = model.predict(test_features)
    
    print(f"Classification Report for {name}: \n {classification_report(y_test, y_pred)                                         

Training SVM...
Classification Report for SVM: 
               precision    recall  f1-score   support

           0       0.80      0.76      0.78       282
           1       0.64      0.82      0.72       359
           2       0.51      0.52      0.52       286
           3       0.40      0.31      0.35       240
           4       0.51      0.52      0.51       225
           5       0.77      0.83      0.80       216
           6       0.46      0.33      0.39       228
           7       0.62      0.62      0.62       252
           9       0.75      0.70      0.73       254
          10       0.53      0.60      0.56       248

    accuracy                           0.61      2590
   macro avg       0.60      0.60      0.60      2590
weighted avg       0.60      0.61      0.60      2590


Training Random Forest...
Classification Report for Random Forest: 
               precision    recall  f1-score   support

           0       0.70      0.73      0.72       282
           1 

In [16]:
model = DecisionTreeClassifier(max_depth = 20)
model.fit(train_features, y_train)
    
y_pred = model.predict(test_features)
    
print(f"Classification Report for DecisionTreeClassifier: \n {classification_report(y_test, y_pred)}")

Classification Report for DecisionTreeClassifier: 
               precision    recall  f1-score   support

           0       0.55      0.55      0.55       282
           1       0.47      0.57      0.51       359
           2       0.28      0.27      0.27       286
           3       0.13      0.13      0.13       240
           4       0.22      0.22      0.22       225
           5       0.56      0.61      0.58       216
           6       0.17      0.13      0.15       228
           7       0.45      0.38      0.41       252
           9       0.46      0.52      0.49       254
          10       0.31      0.29      0.30       248

    accuracy                           0.38      2590
   macro avg       0.36      0.37      0.36      2590
weighted avg       0.37      0.38      0.37      2590



TypeError: ignored

In [19]:
with open("DecisionTreeClassifier.p", "wb") as f:
    pickle.dump(model, f)

In [20]:
model = LogisticRegression()
model.fit(train_features, y_train)
    
y_pred = model.predict(test_features)
    
print(f"Classification Report for LogisticRegression: \n {classification_report(y_test, y_pred)}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report for LogisticRegression: 
               precision    recall  f1-score   support

           0       0.74      0.69      0.71       282
           1       0.45      0.57      0.50       359
           2       0.17      0.11      0.13       286
           3       0.15      0.12      0.14       240
           4       0.34      0.42      0.38       225
           5       0.64      0.73      0.68       216
           6       0.15      0.07      0.10       228
           7       0.40      0.34      0.37       252
           9       0.55      0.62      0.58       254
          10       0.30      0.44      0.36       248

    accuracy                           0.42      2590
   macro avg       0.39      0.41      0.39      2590
weighted avg       0.39      0.42      0.40      2590



In [21]:
 with open("LogisticRegressionmodel.p", "wb") as f:
    pickle.dump(model, f)

In [25]:
model = SVC(kernel='rbf', C=3)
model.fit(train_features, y_train)
    
y_pred = model.predict(test_features)
    
print(f"Classification Report for SVC: \n {classification_report(y_test, y_pred)}")

Classification Report for SVC: 
               precision    recall  f1-score   support

           0       0.83      0.77      0.80       282
           1       0.65      0.83      0.73       359
           2       0.47      0.47      0.47       286
           3       0.40      0.32      0.36       240
           4       0.49      0.48      0.48       225
           5       0.73      0.84      0.78       216
           6       0.45      0.31      0.36       228
           7       0.62      0.60      0.61       252
           9       0.77      0.69      0.73       254
          10       0.51      0.60      0.55       248

    accuracy                           0.60      2590
   macro avg       0.59      0.59      0.59      2590
weighted avg       0.60      0.60      0.59      2590



In [26]:
 with open("SVC.p", "wb") as f:
    pickle.dump(model, f)

In [27]:
model = RandomForestClassifier(n_estimators= 100)
model.fit(train_features, y_train)
    
y_pred = model.predict(test_features)
    
print(f"Classification Report for RandomForestClassifier: \n {classification_report(y_test, y_pred)}")

Classification Report for RandomForestClassifier: 
               precision    recall  f1-score   support

           0       0.74      0.73      0.73       282
           1       0.60      0.84      0.70       359
           2       0.38      0.50      0.43       286
           3       0.28      0.18      0.22       240
           4       0.47      0.41      0.44       225
           5       0.69      0.82      0.75       216
           6       0.40      0.13      0.19       228
           7       0.69      0.63      0.66       252
           9       0.76      0.67      0.72       254
          10       0.51      0.59      0.55       248

    accuracy                           0.57      2590
   macro avg       0.55      0.55      0.54      2590
weighted avg       0.55      0.57      0.55      2590



In [28]:
 with open("RandomForestClassifier.p", "wb") as f:
    pickle.dump(model, f)

In [29]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(train_features, y_train)
    
y_pred = model.predict(test_features)
    
print(f"Classification Report for KNeighborsClassifier: \n {classification_report(y_test, y_pred)}")

Classification Report for KNeighborsClassifier: 
               precision    recall  f1-score   support

           0       0.82      0.55      0.66       282
           1       0.21      0.93      0.34       359
           2       0.20      0.01      0.01       286
           3       0.38      0.01      0.02       240
           4       0.76      0.06      0.11       225
           5       0.59      0.63      0.61       216
           6       0.44      0.02      0.03       228
           7       0.48      0.35      0.41       252
           9       0.48      0.57      0.52       254
          10       0.65      0.09      0.16       248

    accuracy                           0.35      2590
   macro avg       0.50      0.32      0.29      2590
weighted avg       0.48      0.35      0.29      2590



In [30]:
 with open("KNeighborsClassifier.p", "wb") as f:
    pickle.dump(model, f)